In [3]:
import pyarrow.parquet as pq
import pandas as pd
import os
import urllib.request
import collections

# Get data


In [26]:
if 'email.zip' not in os.listdir('data/'):
    dataPath = "https://www.kaggle.com/karthickveerakumar/spam-filter/downloads/spam-filter.zip/1"
#     urllib.request.urlretrieve(dataPath,'data/email.zip')
    print('email.zip is downloaded in data folder')
#     os.system("unzip -a -n data/email.zip -d data/")
    print('email.zip is successfully unziped')

email.zip is downloaded in data folder
email.zip is successfully unziped


## Deal with parquet big data file

In [27]:
dataFolder = "data/dataset.parquet/"
File_list = os.listdir(dataFolder)
File_list = [ dataFolder + x for x in File_list if x.endswith('parquet')] 
File_list

['data/dataset.parquet/part-00004-tid-6056519413201330783-119422a4-5c84-4461-ba64-4a5e2a1d4acd-33-c000.snappy.parquet',
 'data/dataset.parquet/part-00002-tid-6056519413201330783-119422a4-5c84-4461-ba64-4a5e2a1d4acd-31-c000.snappy.parquet',
 'data/dataset.parquet/part-00007-tid-6056519413201330783-119422a4-5c84-4461-ba64-4a5e2a1d4acd-36-c000.snappy.parquet',
 'data/dataset.parquet/part-00001-tid-6056519413201330783-119422a4-5c84-4461-ba64-4a5e2a1d4acd-30-c000.snappy.parquet',
 'data/dataset.parquet/part-00003-tid-6056519413201330783-119422a4-5c84-4461-ba64-4a5e2a1d4acd-32-c000.snappy.parquet',
 'data/dataset.parquet/part-00000-tid-6056519413201330783-119422a4-5c84-4461-ba64-4a5e2a1d4acd-29-c000.snappy.parquet',
 'data/dataset.parquet/part-00006-tid-6056519413201330783-119422a4-5c84-4461-ba64-4a5e2a1d4acd-35-c000.snappy.parquet']

# use pyarrow to read

In [28]:
df_all = pd.DataFrame()
for file in File_list:
    tmp = pq.read_table(file).to_pandas()

    ### get positive data, 0.1% data 
    pos = tmp[tmp['label']==1] 
    ### sample negative data 
    neg = tmp.sample(n=pos.shape[0])
    df_tmp = pd.concat([pos,neg])
    df_all = pd.concat([df_tmp,df_all])
    print(df_all.shape)

collections.Counter(df_all.label)

(118, 503)
(232, 503)
(240, 503)
(360, 503)
(496, 503)
(628, 503)
(736, 503)


Counter({1: 368, 0: 368})

dataset = pq.ParquetDataset(File_list)
table = dataset.read().to_pandas()

In [31]:
from sklearn.decomposition import PCA

my_model = PCA(n_components=0.99, svd_solver='full')
eg = my_model.fit_transform(df_all)

In [34]:
eg.shape

(736, 2)

## NLP part


#### Read raw data

In [4]:
df = pd.read_csv("data/emails.csv") ### download from kaggle

emails = df.text.values

In [5]:
emails[1]

'Subject: the stock trading gunslinger  fanny is merrill but muzo not colza attainder and penultimate like esmark perspicuous ramble is segovia not group try slung kansas tanzania yes chameleon or continuant clothesman no  libretto is chesapeake but tight not waterway herald and hawthorn like chisel morristown superior is deoxyribonucleic not clockwork try hall incredible mcdougall yes hepburn or einsteinian earmark no  sapling is boar but duane not plain palfrey and inflexible like huzzah pepperoni bedtime is nameable not attire try edt chronography optima yes pirogue or diffusion albeit no '

#### NLTK

In [6]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity 
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

/Users/zili/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/zili/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/zili/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
sentim_analyzer = SentimentAnalyzer()
vectorizer = TfidfVectorizer("english") ### check tf-idf theory, input: arr of string, output: feature arr, column: tf-idf feature


# Feature engineer steps
#### 1 insight
#### 2 summarization
#### 3 remove puntucation
#### 4 remove stop words
#### 5 remove stemmization, lemmatization

#### 6 dimension reduction
#### 7 train test split

###### others features: n-gram

### Generate insight

In [14]:
def pre_process(text):
    
    ### Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    ### Remove stops words
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    ### Remove lemmatization
    wnl = nltk.WordNetLemmatizer()
    text = list(map(lambda x:wnl.lemmatize(x),text))
    ### Remove stemmization
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    print('done')
    return words

In [15]:
emails[0]

"Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordability : your  ma

In [16]:
emails = np.array(list(map(lambda x:pre_process(x),emails)))

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [ ]:
emails

In [17]:
features = vectorizer.fit_transform(emails)

In [18]:
features.toarray().shape

(5728, 29113)

### Summarization

In [19]:
from gensim.summarization.summarizer import summarize

In [20]:
emails[0]

'subject natur irresist corpor ident lt realli hard recollect compani market full suqgest inform isoverwhelminq good catchi logo stylish statloneri outstand websit make task much easier promis havinq order iogo compani automaticaili becom world ieader isguit ciear without good product effect busi organ practic aim hotat nowaday market promis market effort becom much effect list clear benefit creativ hand made origin logo special done reflect distinct compani imag conveni logo stationeri provid format easi use content manag system letsyou chang websit content even structur prompt see logo draft within three busi day afford market break make gap budget 100 satisfact guarante provid unlimit amount chang extra fee surethat love result collabor look portfolio interest '

In [21]:
summarize(df['text'].values[0])

'Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .'

In [22]:
def summary(string,**kwargs):
    try:
        summarized = summarize(string,**kwargs)
    except:
        return string
    return summarized

### data manipulation

In [23]:
emailSummary = list(map(lambda x:summary(x),df['text'].values))
newEmails = list(map(lambda x:pre_process(x),emailSummary))

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [25]:
newFeatures = vectorizer.fit_transform(emailSummary)

In [26]:
newFeatures.toarray().shape

(5728, 17836)

### dimension reduction

In [27]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
# pca = PCA(n_components=500, svd_solver='full')
pca = TruncatedSVD(500)

In [28]:
type(newFeatures)

scipy.sparse.csr.csr_matrix

In [29]:
reducedFeatures = pca.fit_transform(newFeatures)

In [30]:
type(reducedFeatures)

numpy.ndarray

In [31]:
reducedFeatures.shape

(5728, 500)

In [32]:
newFeatures = reducedFeatures

# Modeling part

### 1 SVC model
### 2 Naive bayes model
### 3 Deep learning model

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
features_train, features_test, labels_train, labels_test = train_test_split(newFeatures, df['spam'], test_size=0.3, random_state=111)

In [35]:
features_train.shape

(4009, 500)

### SVC model

In [36]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(features_train, labels_train)
prediction = svc.predict(features_test)
print("accuracy",accuracy_score(labels_test,prediction))
confusion_matrix(labels_test,prediction)

accuracy 0.891215823152996


array([[1305,   13],
       [ 174,  227]])

### Naive bayes

from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=0.2)
mnb.fit(features_train, labels_train)
prediction = mnb.predict(features_test)
print("accuracy",accuracy_score(labels_test,prediction))
confusion_matrix(labels_test,prediction)

### DNN

In [38]:
newFeatures.shape

(5728, 500)

In [39]:
features_test.shape

(1719, 500)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()

model.add(Dense(12, input_dim=500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
new_labels_tarin= labels_train

model.fit(features_train, pd.Series(new_labels_tarin), epochs=150, batch_size=10)
predictions = model.predict(features_test)
rounded = [round(x[0]) for x in predictions]
from sklearn.metrics import accuracy_score

/Users/zili/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Epoch 1/150
4009/4009 [==============================] - 1s 235us/step - loss: 0.4071 - acc: 0.8114
Epoch 2/150
4009/4009 [==============================] - 1s 156us/step - loss: 0.2030 - acc: 0.9114
Epoch 3/150
4009/4009 [==============================] - 1s 170us/step - loss: 0.1641 - acc: 0.9264
Epoch 4/150
4009/4009 [==============================] - 1s 163us/step - loss: 0.1492 - acc: 0.9272
Epoch 5/150
4009/4009 [==============================] - 1s 168us/step - loss: 0.1403 - acc: 0.9329
Epoch 6/150
4009/4009 [==============================] - 1s 147us/step - loss: 0.1361 - acc: 0.9334
Epoch 7/150
4009/4009 [==============================] - 1s 143us/step - loss: 0.1280 - acc: 0.9344
Epoch 8/150
4009/4009 [==============================] - 1s 153us/step - loss: 0.1288 - acc: 0.9307
Epoch 9/150
4009/4009 [==============================] - 1s 153us/step - loss: 0.1245 - acc: 0.9364 0s - loss: 0.1268 - acc: 0.93
Epoch 10/150
4009/4009 [==============================] - 1s 153us/ste

In [112]:
new_labels_test=[]
for i in labels_test:
    if i == "ham":
        new_labels_test.append(1.0)
    else:
        new_labels_test.append(0.0)
accuracy_score(new_labels_test,rounded)

0.820826061663758

In [115]:
confusion_matrix(labels_test,prediction)

array([[1310,    8],
       [ 168,  233]])